# Feedforward Network

Quelle: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py#L37-L49

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.01

In [5]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
transform=transforms.ToTensor())

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
shuffle=False)

In [7]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [19]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [29]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [100/600], Loss: 0.2928
Epoch [1/10], Step [200/600], Loss: 0.3585
Epoch [1/10], Step [300/600], Loss: 0.3058
Epoch [1/10], Step [400/600], Loss: 0.3658
Epoch [1/10], Step [500/600], Loss: 0.3476
Epoch [1/10], Step [600/600], Loss: 0.3202
Epoch [2/10], Step [100/600], Loss: 0.1369
Epoch [2/10], Step [200/600], Loss: 0.2795
Epoch [2/10], Step [300/600], Loss: 0.2159
Epoch [2/10], Step [400/600], Loss: 0.2928
Epoch [2/10], Step [500/600], Loss: 0.3407
Epoch [2/10], Step [600/600], Loss: 0.2133
Epoch [3/10], Step [100/600], Loss: 0.2860
Epoch [3/10], Step [200/600], Loss: 0.3560
Epoch [3/10], Step [300/600], Loss: 0.2956
Epoch [3/10], Step [400/600], Loss: 0.3333
Epoch [3/10], Step [500/600], Loss: 0.2459
Epoch [3/10], Step [600/600], Loss: 0.2372
Epoch [4/10], Step [100/600], Loss: 0.2749
Epoch [4/10], Step [200/600], Loss: 0.2338
Epoch [4/10], Step [300/600], Loss: 0.3565
Epoch [4/10], Step [400/600], Loss: 0.3115
Epoch [4/10], Step [500/600], Loss: 0.2072
Epoch [4/10

In [25]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 92.25 %


In [12]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')